# Identity, Immutability, and .NET Interactive Notebooks

Michael L Perry

> @michaellperry

Code posted to https://github.com/michaellperry/identity-immutability


## Location-Specific Identity

Let's get an instance of PostgreSQL running in Docker:

```
docker run --name postgres -p5432:5432 -e POSTGRES_PASSWORD=mypw -d postgres
```

Connect to the container using this command.

```
docker exec -it postgres /bin/bash
```

Once connected, you can start the PostgreSQL client.

```
psql -U postgres
```

Postgres has an object called a Sequence. It's entire purpose is to generate increasing numbers. Every time someone asks for a number from the sequence, they are guaranteed to get a number different -- and bigger -- than any number that anyone else has seen.

Create a sequence like this:

```
CREATE SEQUENCE id;
```

Then get the next number like this:

```
SELECT nextval('id');
```

To quit the PostgreSQL client, just type `\q` and press enter.
Then type `exit` and press enter to log out of the Docker container.

You can also do that in a client application using a Postgres driver.
This code will install a driver via a NuGet package and then get the next ID from the sequence.

In [ ]:
#r "nuget: Npgsql, 5.0.7"

using Npgsql;

var connString = "Host=localhost;Username=postgres;Password=mypw;Database=postgres";

var conn = new NpgsqlConnection(connString);
await conn.OpenAsync();

var comm = new NpgsqlCommand("SELECT nextval('id')", conn);
var nextId = await comm.ExecuteScalarAsync();
await comm.DisposeAsync();

await conn.DisposeAsync();

nextId

Installed Packages Npgsql, 5.0.7

4

In [ ]:
Guid.NewGuid()

4babca6d-41a1-4df2-a2bf-3786d2630347

In [ ]:
using System.Security.Cryptography;

var rsa = RSA.Create(2048);
byte[] bytes = rsa.ExportRSAPublicKey();
string publicKey = Convert.ToBase64String(bytes);
publicKey

MIIBCgKCAQEA807cb6nBI+1UGxfNfIdnW4uK7n8ncWavgCWCIGY0hdtFSZFT2l6asC6MUMIoQrvFCxtv0rAQpP12aDynQRE8MBU40rmCw3/s3Zp8F4b5r//GcRjusACK6g0+d775iw683kcU7iLFviMk3SB02VNS3ZNBMPdhLxttEM8rZFSkxg7kH45I6TRq4xZqe8JFij9/VETFGlm+IGmE4U7r8fUAzCEvN8sh6xPYYkAzRIp8TC21ClRwFJiSl/jyfiLISyHaqJXNsTVNKK7S/Tfdwz1TujjvyWYhWig93tAvQ+Nad2N/H8CfLFbJl9FNhXoTw9KdXgOMKO9vtevTQ/GwUoR1MQIDAQAB

In [ ]:
byte[] key = rsa.ExportRSAPrivateKey();
Convert.ToBase64String(key)

MIIEogIBAAKCAQEA807cb6nBI+1UGxfNfIdnW4uK7n8ncWavgCWCIGY0hdtFSZFT2l6asC6MUMIoQrvFCxtv0rAQpP12aDynQRE8MBU40rmCw3/s3Zp8F4b5r//GcRjusACK6g0+d775iw683kcU7iLFviMk3SB02VNS3ZNBMPdhLxttEM8rZFSkxg7kH45I6TRq4xZqe8JFij9/VETFGlm+IGmE4U7r8fUAzCEvN8sh6xPYYkAzRIp8TC21ClRwFJiSl/jyfiLISyHaqJXNsTVNKK7S/Tfdwz1TujjvyWYhWig93tAvQ+Nad2N/H8CfLFbJl9FNhXoTw9KdXgOMKO9vtevTQ/GwUoR1MQIDAQABAoIBAGTvrltUnR9+cYnb0g97TNGZjkVq0r4kz2HmLZmrTNbLJG9krtjErn+wj91g7rrgktoMw9Mv33fS/rA3mVtQ/xsjcrNY6nqYM3ergxdvk14EShrZMQwq7XYzkFq1C2k47/l2m6Q/ILHEeqKRuGgAPTxWvpDFz7dvNUvn5iR1SPYwoDGRvU13zKnVaLbo4p3z57kgp8jytOVMrv4kJgvpXUKyiOoto+24K1Zy7ELzJ1yreDN4THQnQrZDQs/1BDipcyPffheKh4NlzgvzqqKyY50q6NiQ6m+YXytzrWEWPpVrAqwiwzy8vvmBPP+oXiq5giZrBOev7TiNtyV/s0Oa+ZECgYEA9cPXUEosV2Du6Cl5e7Vh+OThzLjqzyjGuuaW0E6qCXB4YL3Po51rGbUTVOJeD3ADcFshPeS2WZb9cs+kzdQ/JaPHxjs5m/jyrkWnrvJLA/E6/lMg5Gtk6HAZIP5DztviC8JBTZB/Fch/4g303Y1gAJ+SXHpZ3C4Y/9ZHlyJ8AlMCgYEA/XDTavnAwkCjP3ckWm7sdjrhP3l6Girhm/cmMnYjv1BcusWMoRxRnmllo9YlP1OIRMUKS0RM0lF9IINLIXfZXk315X9RYspLU74QGh3YmnBO

In [ ]:
record Person(string publicKey);


In [ ]:

var a = new Person("--- PUB ---");
var b = new Person("--- PUB ---");

$"b = {b.GetHashCode()} a = {a.GetHashCode()}"

b = 100267936 a = 100267936

In [ ]:
object.ReferenceEquals(a, b)

False

In [ ]:
record PersonName(Person person, string value);

In [ ]:
var recordsOfTypePerson = new HashSet<Person>();
var recordsOfTypePersonName = new HashSet<PersonName>();

var george = new Person(publicKey);
var georgesName = new PersonName(george, "George");

recordsOfTypePerson.Add(george);
recordsOfTypePersonName.Add(georgesName);

var sally = new Person("---dkfjhsdkjdhf---");
var sallysName = new PersonName(sally, "Sally");

recordsOfTypePerson.Add(sally);
recordsOfTypePersonName.Add(sallysName);

var result =
    from personName in recordsOfTypePersonName
    where personName.person.Equals(george)
    select personName;

result

index person value 0 publicKey MIIBCgKCAQEA807cb6nBI+1UGxfNfIdnW4uK7n8ncWavgCWCIGY0hdtFSZFT2l6asC6MUMIoQrvFCxtv0rAQpP12aDynQRE8MBU40rmCw3/s3Zp8F4b5r//GcRjusACK6g0+d775iw683kcU7iLFviMk3SB02VNS3ZNBMPdhLxttEM8rZFSkxg7kH45I6TRq4xZqe8JFij9/VETFGlm+IGmE4U7r8fUAzCEvN8sh6xPYYkAzRIp8TC21ClRwFJiSl/jyfiLISyHaqJXNsTVNKK7S/Tfdwz1TujjvyWYhWig93tAvQ+Nad2N/H8CfLFbJl9FNhXoTw9KdXgOMKO9vtevTQ/GwUoR1MQIDAQAB George

In [ ]:
#r "nuget: Jinaga.UnitTest, 0.1.1"

using Jinaga;
using Jinaga.UnitTest;

var j = JinagaTest.Create();

[FactType("Jinaga.User")]
record Person(string publicKey);

[FactType("Person.Name")]
record PersonName(Person person, string value, PersonName[] prior);


Installed Packages Jinaga.UnitTest, 0.1.1

In [ ]:
#r "nuget: Jinaga.Graphviz, 0.1.1"



Installed Packages Jinaga.Graphviz, 0.1.1

In [ ]:

var george = await j.Fact(new Person("--- PUB ---"));
var georgesName = await j.Fact(new PersonName(george, "George", new PersonName[] {}));
await j.Fact(new PersonName(george, "Jorge", new [] { georgesName }));

var namesOfPerson = Given<Person>.Match((person, facts) =>
    from personName in facts.OfType<PersonName>()
    where personName.person == george
    where !(
        from next in facts.OfType<PersonName>()
        where next.prior.Contains(personName)
        select next
    ).Any()
    select personName
);
var georgesNames = await j.Query(george, namesOfPerson);

//georgesNames.Count()
Jinaga.Graphviz.Renderer.RenderFacts(georgesNames)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.48.0 (20210717.1556)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 3NsFsBQo2URgAC3KOQgRp0054c1iK9F1KzMQGM78CmaqFz3GkQNub0KaHQBmo6bDgRzIEPF4dQfibxOwJbUgcA== -->
 
 3NsFsBQo2URgAC3KOQgRp0054c1iK9F1KzMQGM78CmaqFz3GkQNub0KaHQBmo6bDgRzIEPF4dQfibxOwJbUgcA== 
 
 Jinaga.User 
 
 publicKey 
 
 "--- PUB ---" 
 
<!-- kxupOOpMqi9u8FHn/n2kai5au7DvP89vJMiaGGWIDjX3waVYqY7R29YJ7Ooit5SRTWDhxTOlosTGLcA/EE8kbg== -->
 
 kxupOOpMqi9u8FHn/n2kai5au7DvP89vJMiaGGWIDjX3waVYqY7R29YJ7Ooit5SRTWDhxTOlosTGLcA/EE8kbg== 
 
 Person.Name 
 
 value 
 
 "George" 
 
<!-- kxupOOpMqi9u8FHn/n2kai5au7DvP89vJMiaGGWIDjX3waVYqY7R29YJ7Ooit5SRTWDhxTOlosTGLcA/EE8kbg==->3NsFsBQo2URgAC3KOQgRp0054c1iK9F1KzMQGM78CmaqFz3GkQNub0KaHQBmo6bDgRzIEPF4dQfibxOwJbUgcA== -->
 
 kxupOOpMqi9u8FHn/n2kai5au7DvP89vJMiaGGWIDjX3waVYqY7R29YJ7Ooit5SRTWDhxTOlosTGLcA/EE8kbg==->3NsFsBQo2URgAC3KOQgRp0054c1iK9F1KzMQGM78CmaqFz3GkQNub0KaHQBmo6bDgRzIEPF4dQfibxOwJbUgcA== 
 
 
 person 
 
<!-- +5u9Q3xKOm82JRW2YnDsogmx22cvcpeduGGq2Ry1WPDre7F/nNKf18hDvOOjLS8vcJnQiLeFbU0IqYNopTqX3A== -->
 
 +5u9Q3xKOm82JRW2YnDsogmx22cvcpeduGGq2Ry1WPDre7F/nNKf18hDvOOjLS8vcJnQiLeFbU0IqYNopTqX3A== 
 
 Person.Name 
 
 value 
 
 "Jorge" 
 
 
<!-- +5u9Q3xKOm82JRW2YnDsogmx22cvcpeduGGq2Ry1WPDre7F/nNKf18hDvOOjLS8vcJnQiLeFbU0IqYNopTqX3A==->3NsFsBQo2URgAC3KOQgRp0054c1iK9F1KzMQGM78CmaqFz3GkQNub0KaHQBmo6bDgRzIEPF4dQfibxOwJbUgcA== -->
 
 +5u9Q3xKOm82JRW2YnDsogmx22cvcpeduGGq2Ry1WPDre7F/nNKf18hDvOOjLS8vcJnQiLeFbU0IqYNopTqX3A==->3NsFsBQo2URgAC3KOQgRp0054c1iK9F1KzMQGM78CmaqFz3GkQNub0KaHQBmo6bDgRzIEPF4dQfibxOwJbUgcA== 
 
 
 person 
 
<!-- +5u9Q3xKOm82JRW2YnDsogmx22cvcpeduGGq2Ry1WPDre7F/nNKf18hDvOOjLS8vcJnQiLeFbU0IqYNopTqX3A==->kxupOOpMqi9u8FHn/n2kai5au7DvP89vJMiaGGWIDjX3waVYqY7R29YJ7Ooit5SRTWDhxTOlosTGLcA/EE8kbg== -->
 
 +5u9Q3xKOm82JRW2YnDsogmx22cvcpeduGGq2Ry1WPDre7F/nNKf18hDvOOjLS8vcJnQiLeFbU0IqYNopTqX3A==->kxupOOpMqi9u8FHn/n2kai5au7DvP89vJMiaGGWIDjX3waVYqY7R29YJ7Ooit5SRTWDhxTOlosTGLcA/EE8kbg== 
 
 
 prior

In [ ]:
namesOfPerson.Pipeline.ToDescriptiveString()

george: Jinaga.User {
    personName: Person.Name = george S.person Person.Name
    N(
        personName: Person.Name {
            next: Person.Name = personName S.prior Person.Name
        }
    )
}


In [ ]:
double AreaOfCircle(double radius) => Math.PI * radius * radius;


class Circle
{
    public double Radius { get; set; }
    public double Area => Math.PI * Radius * Radius;

    public Circle(double radius)
    {
        Radius = radius;
    }
}

// Michael Perry is a C# developer, and he is a fan of the C# language.
// Jon Skeet is a StackOverflow contributor, and he is a fan of the StackOverflow community.